# Lab 1.3.2: Matrix Multiplication

**Module:** 1.3 - CUDA Python & GPU Programming  
**Time:** 3 hours  
**Difficulty:** ⭐⭐⭐⭐ (Advanced)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand why matrix multiplication is the heart of deep learning
- [ ] Implement naive matrix multiplication on GPU
- [ ] Master the **tiled algorithm** using shared memory
- [ ] Understand memory coalescing and bank conflicts
- [ ] Achieve performance within **2x of cuBLAS** (NVIDIA's optimized library)

---

## 📚 Prerequisites

- Completed: Lab 1.3.1 (Parallel Reduction)
- Knowledge of: Shared memory, thread synchronization

---

## 🌍 Real-World Context

**Matrix multiplication is literally 90%+ of deep learning compute.**

Every neural network operation boils down to matmul:
- **Linear layers:** `y = Wx + b` → Matrix multiplication
- **Attention mechanism:** `softmax(QK^T)V` → Three matrix multiplications!
- **Convolutions:** Can be expressed as matrix multiplication (im2col)
- **Embeddings:** Sparse matrix multiplication

**The numbers are staggering:**
- GPT-3 forward pass: ~350 trillion floating-point operations (FLOPs)
- Almost all of that is matrix multiplication
- Training GPT-3 required ~3,000 PFLOP-days of compute

Your DGX Spark can perform **~100 TFLOPS in BF16**. Understanding how matrix multiplication uses this power is essential!

---

## 🧒 ELI5: Matrix Multiplication on GPU

> **Imagine you're organizing a giant wedding reception** with 1000 tables and 1000 guests.
>
> Each guest needs to receive food from multiple serving stations. The naive approach:
> - Each server walks to every table one by one (slow!)
> - They make 1000 trips, and there are 1000 servers = 1,000,000 trips!
>
> **The tiled approach:**
> - Divide the venue into "zones" (tiles)
> - Each zone has a small buffet table (shared memory)
> - Servers load food onto the local buffet once
> - All guests in that zone can quickly grab from the buffet
> - Then move to the next zone
>
> **In matrix terms:**
> - Each output element needs data from an entire row and column
> - Loading from main memory is slow (the trips across the venue)
> - We load a "tile" of data into shared memory (the local buffet)
> - Multiple threads use that same data (guests at that table)
> - This is called **data reuse** - the key to GPU performance!

```
Matrix A (M×K)         Matrix B (K×N)         Matrix C (M×N)
┌─────────────┐       ┌─────────────┐       ┌─────────────┐
│ █ █ █ █ █ █ │       │ █ █ │ │ │ │ │       │ ✖ │ │ │ │ │ │
│ │ │ │ │ │ │ │   ×   │ █ █ │ │ │ │ │   =   │ │ │ │ │ │ │ │
│ │ │ │ │ │ │ │       │ █ █ │ │ │ │ │       │ │ │ │ │ │ │ │
│ │ │ │ │ │ │ │       │ █ █ │ │ │ │ │       │ │ │ │ │ │ │ │
└─────────────┘       └─────────────┘       └─────────────┘
Row 0 of A (█)    × Column 0 of B (█)    = Element C[0,0] (✖)
```

---

## Part 0: Environment Setup

In [ ]:
import numpy as np
import time
from typing import Tuple
import warnings
warnings.filterwarnings('ignore')

from numba import cuda, float32
import numba

# Verify CUDA
print("🔧 CUDA Environment")
print("="*50)
print(f"Numba version: {numba.__version__}")
print(f"CUDA available: {cuda.is_available()}")

if cuda.is_available():
    device = cuda.get_current_device()
    print(f"Device: {device.name}")
    print(f"Shared memory per block: {device.MAX_SHARED_MEMORY_PER_BLOCK / 1024:.1f} KB")
    
    # Calculate max tile size based on shared memory
    # We need 2 tiles (for A and B), each TILE_SIZE² × 4 bytes (float32)
    max_shared = device.MAX_SHARED_MEMORY_PER_BLOCK
    max_tile_size = int(np.sqrt(max_shared / 8))  # 8 = 2 tiles × 4 bytes
    print(f"Max tile size (theoretical): {max_tile_size}")
    print("\n✅ Ready for matrix multiplication!")

---

## Part 1: CPU and NumPy Baselines

First, let's understand how matrix multiplication works and establish baselines.

In [ ]:
def matmul_cpu_naive(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    """
    Naive CPU matrix multiplication using triple nested loop.
    
    This is O(M × N × K) - cubic complexity!
    """
    M, K = A.shape
    K2, N = B.shape
    assert K == K2, "Inner dimensions must match!"
    
    C = np.zeros((M, N), dtype=A.dtype)
    
    for i in range(M):       # For each row of A
        for j in range(N):   # For each column of B
            for k in range(K):  # Dot product
                C[i, j] += A[i, k] * B[k, j]
    
    return C


# Small test to verify correctness
A_small = np.array([[1, 2], [3, 4]], dtype=np.float32)
B_small = np.array([[5, 6], [7, 8]], dtype=np.float32)

result_naive = matmul_cpu_naive(A_small, B_small)
result_numpy = A_small @ B_small  # NumPy's optimized matmul

print("Matrix A:")
print(A_small)
print("\nMatrix B:")
print(B_small)
print("\nA × B (naive):")
print(result_naive)
print("\nA × B (NumPy):")
print(result_numpy)
print(f"\n✅ Results match: {np.allclose(result_naive, result_numpy)}")

### Understanding the Math

For `C = A × B` where A is M×K and B is K×N:

```
C[i,j] = Σ(k=0 to K-1) A[i,k] × B[k,j]
```

Each element of C requires **K multiplications and K-1 additions** (a dot product).

For M×N outputs, total operations: **2 × M × N × K FLOPs** (multiply and add).

In [ ]:
# Benchmark on larger matrices
np.random.seed(42)

# Test sizes
sizes = [(64, 64), (256, 256), (512, 512), (1024, 1024)]

print("⏱️  NumPy Matmul Baseline (uses BLAS/MKL internally)")
print("="*60)
print(f"{'Size':<15} {'Time (ms)':<15} {'GFLOPS':<15}")
print("-"*45)

for M, N in sizes:
    K = M  # Square matrices
    A = np.random.randn(M, K).astype(np.float32)
    B = np.random.randn(K, N).astype(np.float32)
    
    # Warm up
    _ = A @ B
    
    # Benchmark
    start = time.perf_counter()
    iterations = 10
    for _ in range(iterations):
        C = A @ B
    elapsed = (time.perf_counter() - start) / iterations
    
    # Calculate GFLOPS: 2*M*N*K operations (multiply-add)
    flops = 2 * M * N * K
    gflops = flops / elapsed / 1e9
    
    print(f"{M}×{N}×{K:<7} {elapsed*1000:<15.3f} {gflops:<15.1f}")

# Store the 1024×1024 for later comparison
A_1024 = np.random.randn(1024, 1024).astype(np.float32)
B_1024 = np.random.randn(1024, 1024).astype(np.float32)
C_reference = A_1024 @ B_1024

---

## Part 2: Naive GPU Matrix Multiplication

**Strategy:** Each thread computes one element of the output matrix C.

```
Thread (i,j) computes C[i,j]:
    sum = 0
    for k = 0 to K-1:
        sum += A[i,k] * B[k,j]
    C[i,j] = sum
```

Simple, but each thread does **K global memory reads for row of A** and **K global memory reads for column of B**. That's 2K reads per thread!

In [ ]:
@cuda.jit
def matmul_naive_kernel(A, B, C):
    """
    Naive GPU matrix multiplication.
    
    Each thread computes one element of C.
    Problem: Lots of redundant global memory access!
    """
    # 2D grid/block indexing
    row = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    col = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    
    M, K = A.shape
    K2, N = B.shape
    
    # Bounds check
    if row < M and col < N:
        # Compute dot product of row of A and column of B
        tmp = 0.0
        for k in range(K):
            tmp += A[row, k] * B[k, col]  # K reads from A, K reads from B
        C[row, col] = tmp


def matmul_gpu_naive(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    """
    Matrix multiplication using naive GPU kernel.
    """
    M, K = A.shape
    K2, N = B.shape
    assert K == K2
    
    # Transfer to GPU
    d_A = cuda.to_device(A)
    d_B = cuda.to_device(B)
    d_C = cuda.device_array((M, N), dtype=np.float32)
    
    # Configure grid: 16×16 threads per block
    threads_per_block = (16, 16)
    blocks_x = (N + 15) // 16
    blocks_y = (M + 15) // 16
    blocks = (blocks_x, blocks_y)
    
    # Launch kernel
    matmul_naive_kernel[blocks, threads_per_block](d_A, d_B, d_C)
    
    return d_C.copy_to_host()


# Test correctness
result_gpu = matmul_gpu_naive(A_small, B_small)
print("Naive GPU result:")
print(result_gpu)
print(f"\n✅ Correct: {np.allclose(result_gpu, result_numpy)}")

In [ ]:
# Benchmark naive GPU
print("\n⏱️  Benchmarking Naive GPU Matmul...")
print("="*60)

# Pre-transfer data
d_A = cuda.to_device(A_1024)
d_B = cuda.to_device(B_1024)
d_C = cuda.device_array((1024, 1024), dtype=np.float32)

threads_per_block = (16, 16)
blocks = (64, 64)

# Warm up
for _ in range(3):
    matmul_naive_kernel[blocks, threads_per_block](d_A, d_B, d_C)
cuda.synchronize()

# Benchmark
iterations = 10
start = time.perf_counter()
for _ in range(iterations):
    matmul_naive_kernel[blocks, threads_per_block](d_A, d_B, d_C)
cuda.synchronize()
time_naive_gpu = (time.perf_counter() - start) / iterations

# Calculate GFLOPS
flops = 2 * 1024 * 1024 * 1024
gflops_naive = flops / time_naive_gpu / 1e9

print(f"Matrix size: 1024×1024")
print(f"Naive GPU time: {time_naive_gpu*1000:.3f} ms")
print(f"Naive GPU GFLOPS: {gflops_naive:.1f}")

# Verify correctness
result = d_C.copy_to_host()
print(f"\n✅ Result correct: {np.allclose(result, C_reference, rtol=1e-4)}")

### 🔍 Why Is Naive GPU Still Slow?

Let's analyze the memory access pattern:

```
For 1024×1024 matrices:
- Total output elements: 1,048,576
- Each element needs: 1024 reads from A + 1024 reads from B = 2048 reads
- Total memory reads: ~2 billion!
- At 273 GB/s (DGX Spark), that's ~30ms just for memory access
```

**The problem:** We're reading the same data over and over!
- Row 0 of A is read by ALL threads computing row 0 of C (1024 times!)
- Column 0 of B is read by ALL threads computing column 0 of C (1024 times!)

**The solution:** Load data into shared memory and reuse it!

---

## Part 3: Tiled Matrix Multiplication

### 🧒 ELI5: Tiling (Again, with Pictures!)

> Imagine you're solving a giant jigsaw puzzle (the output matrix C).
>
> **Naive approach:** For each puzzle piece, you look at the entire picture on box A and box B, comparing every single feature.
>
> **Tiled approach:** 
> 1. Cut the puzzle into small "tiles" (e.g., 16×16 pieces each)
> 2. For each tile of the output:
>    - Bring a small section of box A's picture to your desk (shared memory)
>    - Bring a small section of box B's picture to your desk
>    - Everyone working on this tile uses these small sections
>    - Then bring the next sections, and so on
> 3. You never need the whole box at once - just small, reusable sections!

### The Algorithm Visualized

```
For each output tile of C (16×16 elements):
┌───────────────┐     ┌───────────────┐     ┌───────────────┐
│   Matrix A    │     │   Matrix B    │     │   Matrix C    │
│ ┌──┬──┬──┬──┐ │     │ ┌──┬──┬──┬──┐ │     │ ┌──┬──┬──┬──┐ │
│ │██│  │  │  │ │     │ │██│  │  │  │ │     │ │✖ │  │  │  │ │
│ ├──┼──┼──┼──┤ │  ×  │ ├──┼──┼──┼──┤ │  =  │ ├──┼──┼──┼──┤ │
│ │  │  │  │  │ │     │ │  │  │  │  │ │     │ │  │  │  │  │ │
│ └──┴──┴──┴──┘ │     │ └──┴──┴──┴──┘ │     │ └──┴──┴──┴──┘ │
└───────────────┘     └───────────────┘     └───────────────┘
   Load tile 0          Load tile 0         Accumulate

       ⬇                    ⬇                   ⬇
   Load tile 1          Load tile 1         Accumulate
       ⬇                    ⬇                   ⬇
      ...                  ...                 ...
       ⬇                    ⬇                   ⬇
   Load tile K/16       Load tile K/16      Final result
```

**Key insight:** The 16×16 tile of A is used by ALL 256 threads computing the output tile. Instead of 256×16 reads, we do 16 reads total!

In [ ]:
# Tile size - must match threads per block
TILE_SIZE = 16

@cuda.jit
def matmul_tiled_kernel(A, B, C):
    """
    Tiled matrix multiplication using shared memory.
    
    Each block computes one 16×16 tile of C.
    Tiles are loaded cooperatively into shared memory for reuse.
    """
    # Shared memory for tiles of A and B
    sA = cuda.shared.array(shape=(TILE_SIZE, TILE_SIZE), dtype=float32)
    sB = cuda.shared.array(shape=(TILE_SIZE, TILE_SIZE), dtype=float32)
    
    # Thread indices
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    # Block indices - which tile of C are we computing?
    bx = cuda.blockIdx.x
    by = cuda.blockIdx.y
    
    # Global row/col this thread is responsible for
    row = by * TILE_SIZE + ty
    col = bx * TILE_SIZE + tx
    
    M, K = A.shape
    K2, N = B.shape
    
    # Accumulator for this thread's output element
    tmp = float32(0.0)
    
    # Number of tiles we need to process
    num_tiles = (K + TILE_SIZE - 1) // TILE_SIZE
    
    for tile_idx in range(num_tiles):
        # Step 1: Cooperatively load tile of A into shared memory
        # Each thread loads one element
        a_col = tile_idx * TILE_SIZE + tx
        if row < M and a_col < K:
            sA[ty, tx] = A[row, a_col]
        else:
            sA[ty, tx] = 0.0  # Padding for boundary tiles
        
        # Step 2: Cooperatively load tile of B into shared memory
        b_row = tile_idx * TILE_SIZE + ty
        if b_row < K and col < N:
            sB[ty, tx] = B[b_row, col]
        else:
            sB[ty, tx] = 0.0
        
        # CRITICAL: Wait for all threads to finish loading!
        cuda.syncthreads()
        
        # Step 3: Compute partial dot product using shared memory
        # This is where the magic happens - we read from fast shared memory!
        for k in range(TILE_SIZE):
            tmp += sA[ty, k] * sB[k, tx]
        
        # CRITICAL: Wait before loading next tile!
        cuda.syncthreads()
    
    # Step 4: Write final result
    if row < M and col < N:
        C[row, col] = tmp


def matmul_gpu_tiled(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    """
    Matrix multiplication using tiled GPU kernel.
    """
    M, K = A.shape
    K2, N = B.shape
    assert K == K2
    
    d_A = cuda.to_device(A)
    d_B = cuda.to_device(B)
    d_C = cuda.device_array((M, N), dtype=np.float32)
    
    threads_per_block = (TILE_SIZE, TILE_SIZE)
    blocks_x = (N + TILE_SIZE - 1) // TILE_SIZE
    blocks_y = (M + TILE_SIZE - 1) // TILE_SIZE
    blocks = (blocks_x, blocks_y)
    
    matmul_tiled_kernel[blocks, threads_per_block](d_A, d_B, d_C)
    
    return d_C.copy_to_host()


# Test correctness
result_tiled = matmul_gpu_tiled(A_small, B_small)
print("Tiled GPU result:")
print(result_tiled)
print(f"\n✅ Correct: {np.allclose(result_tiled, result_numpy)}")

In [ ]:
# Benchmark tiled GPU
print("\n⏱️  Benchmarking Tiled GPU Matmul...")
print("="*60)

# Pre-transfer data
d_A = cuda.to_device(A_1024)
d_B = cuda.to_device(B_1024)
d_C = cuda.device_array((1024, 1024), dtype=np.float32)

threads_per_block = (TILE_SIZE, TILE_SIZE)
blocks = (64, 64)  # 1024 / 16 = 64

# Warm up
for _ in range(3):
    matmul_tiled_kernel[blocks, threads_per_block](d_A, d_B, d_C)
cuda.synchronize()

# Benchmark
iterations = 10
start = time.perf_counter()
for _ in range(iterations):
    matmul_tiled_kernel[blocks, threads_per_block](d_A, d_B, d_C)
cuda.synchronize()
time_tiled_gpu = (time.perf_counter() - start) / iterations

gflops_tiled = flops / time_tiled_gpu / 1e9

print(f"Matrix size: 1024×1024")
print(f"\n{'Method':<20} {'Time (ms)':<15} {'GFLOPS':<15} {'Speedup':<15}")
print("-"*65)
print(f"{'Naive GPU':<20} {time_naive_gpu*1000:<15.3f} {gflops_naive:<15.1f} {'1.0x':<15}")
print(f"{'Tiled GPU':<20} {time_tiled_gpu*1000:<15.3f} {gflops_tiled:<15.1f} {f'{time_naive_gpu/time_tiled_gpu:.1f}x':<15}")

# Verify correctness
result = d_C.copy_to_host()
print(f"\n✅ Result correct: {np.allclose(result, C_reference, rtol=1e-4)}")

### 🔍 Why Is Tiled Faster?

**Memory Access Analysis:**

| Metric | Naive | Tiled |
|--------|-------|-------|
| Global reads per element | 2K | 2K/TILE_SIZE |
| For 1024×1024, TILE=16 | 2×1024 = 2048 | 2×64 = 128 |
| Total global reads | 2.1 billion | 134 million |
| **Reduction factor** | - | **16x fewer!** |

The tiles are stored in shared memory (~100x faster than global memory), so the inner loop runs at near-theoretical peak speed!

---

## Part 4: Comparing with cuBLAS

cuBLAS is NVIDIA's highly optimized library that squeezes every last FLOP from the GPU. Let's see how we compare!

In [ ]:
# We can use CuPy's BLAS which wraps cuBLAS
try:
    import cupy as cp
    HAS_CUPY = True
    print("✅ CuPy available - will use cuBLAS for comparison")
except ImportError:
    HAS_CUPY = False
    print("⚠️ CuPy not available - skipping cuBLAS comparison")
    print("   ⚠️  Note: 'pip install cupy-cuda12x' installs x86 wheels (won't work on DGX Spark ARM64!)")
    print("   ✅ Use NGC container where CuPy is pre-installed and optimized for ARM64")

In [ ]:
if HAS_CUPY:
    # Convert to CuPy arrays
    cp_A = cp.asarray(A_1024)
    cp_B = cp.asarray(B_1024)
    
    # Warm up
    for _ in range(3):
        cp_C = cp_A @ cp_B
    cp.cuda.Stream.null.synchronize()
    
    # Benchmark cuBLAS
    start = time.perf_counter()
    iterations = 10
    for _ in range(iterations):
        cp_C = cp_A @ cp_B
    cp.cuda.Stream.null.synchronize()
    time_cublas = (time.perf_counter() - start) / iterations
    
    gflops_cublas = flops / time_cublas / 1e9
    
    print("\n" + "="*70)
    print("📊 COMPLETE BENCHMARK: 1024×1024 Matrix Multiplication")
    print("="*70)
    print(f"\n{'Method':<20} {'Time (ms)':<12} {'GFLOPS':<12} {'vs Naive':<12} {'vs cuBLAS':<12}")
    print("-"*70)
    print(f"{'Naive GPU':<20} {time_naive_gpu*1000:<12.3f} {gflops_naive:<12.1f} {'1.0x':<12} {f'{time_naive_gpu/time_cublas:.1f}x slower':<12}")
    print(f"{'Tiled GPU':<20} {time_tiled_gpu*1000:<12.3f} {gflops_tiled:<12.1f} {f'{time_naive_gpu/time_tiled_gpu:.1f}x faster':<12} {f'{time_tiled_gpu/time_cublas:.1f}x':<12}")
    print(f"{'cuBLAS':<20} {time_cublas*1000:<12.3f} {gflops_cublas:<12.1f} {f'{time_naive_gpu/time_cublas:.1f}x faster':<12} {'1.0x':<12}")
    
    # Calculate how close we are to cuBLAS
    pct_of_cublas = gflops_tiled / gflops_cublas * 100
    print(f"\n🎯 Our tiled implementation achieves {pct_of_cublas:.1f}% of cuBLAS performance!")
    
    if pct_of_cublas > 50:
        print("   ✅ Great work! You're within 2x of the world's best!")
    else:
        print("   💡 There's room for optimization. Try larger tile sizes!")

---

## Part 5: Advanced Optimizations (Overview)

Our tiled implementation is good, but cuBLAS uses several additional tricks:

### 1. Larger Tiles and Register Blocking
```python
# Each thread computes a 4×4 or 8×8 block of output
# Uses more registers, fewer shared memory accesses
```

### 2. Double Buffering (Prefetching)
```python
# While computing with tile N, load tile N+1
# Hides memory latency
```

### 3. Avoiding Bank Conflicts
```python
# Shared memory has 32 banks
# If multiple threads access the same bank = conflict = serialization
# Solution: Add padding to shared memory arrays
sA = cuda.shared.array((TILE_SIZE, TILE_SIZE + 1), ...)  # +1 padding
```

### 4. Tensor Cores
```python
# On Blackwell (your DGX Spark), Tensor Cores can do:
# 16×16×16 matrix multiply in ONE instruction!
# That's 4,096 FMAs per Tensor Core per clock!
# cuBLAS automatically uses these for supported data types
```

Let's implement bank conflict avoidance:

In [ ]:
# Bank conflict-free version
TILE_SIZE_PADDED = TILE_SIZE + 1  # Padding to avoid bank conflicts

@cuda.jit
def matmul_tiled_noconflict_kernel(A, B, C):
    """
    Tiled matmul with bank conflict avoidance.
    
    Adding +1 to the second dimension of shared arrays
    prevents bank conflicts when accessing columns.
    """
    # Note the +1 padding!
    sA = cuda.shared.array(shape=(TILE_SIZE, TILE_SIZE_PADDED), dtype=float32)
    sB = cuda.shared.array(shape=(TILE_SIZE, TILE_SIZE_PADDED), dtype=float32)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bx = cuda.blockIdx.x
    by = cuda.blockIdx.y
    
    row = by * TILE_SIZE + ty
    col = bx * TILE_SIZE + tx
    
    M, K = A.shape
    K2, N = B.shape
    
    tmp = float32(0.0)
    num_tiles = (K + TILE_SIZE - 1) // TILE_SIZE
    
    for tile_idx in range(num_tiles):
        a_col = tile_idx * TILE_SIZE + tx
        if row < M and a_col < K:
            sA[ty, tx] = A[row, a_col]
        else:
            sA[ty, tx] = 0.0
        
        b_row = tile_idx * TILE_SIZE + ty
        if b_row < K and col < N:
            sB[ty, tx] = B[b_row, col]
        else:
            sB[ty, tx] = 0.0
        
        cuda.syncthreads()
        
        for k in range(TILE_SIZE):
            tmp += sA[ty, k] * sB[k, tx]
        
        cuda.syncthreads()
    
    if row < M and col < N:
        C[row, col] = tmp


# Benchmark the optimized version
d_C = cuda.device_array((1024, 1024), dtype=np.float32)

# Warm up
for _ in range(3):
    matmul_tiled_noconflict_kernel[blocks, threads_per_block](d_A, d_B, d_C)
cuda.synchronize()

# Benchmark
start = time.perf_counter()
for _ in range(iterations):
    matmul_tiled_noconflict_kernel[blocks, threads_per_block](d_A, d_B, d_C)
cuda.synchronize()
time_noconflict = (time.perf_counter() - start) / iterations

gflops_noconflict = flops / time_noconflict / 1e9

print(f"\nBank conflict-free version:")
print(f"  Time: {time_noconflict*1000:.3f} ms")
print(f"  GFLOPS: {gflops_noconflict:.1f}")
print(f"  Improvement: {time_tiled_gpu/time_noconflict:.2f}x vs basic tiled")

---

## ⚠️ Common Mistakes

### Mistake 1: Wrong Tile Loading Pattern

In [ ]:
# ❌ WRONG: Loading A incorrectly
# sA[ty, tx] = A[ty, tile_idx * TILE_SIZE + tx]  # Wrong row!

# ✅ CORRECT: Use global row index
# sA[ty, tx] = A[row, tile_idx * TILE_SIZE + tx]

print("💡 Common confusion:")
print("   - ty, tx are LOCAL indices (0-15 for 16x16 tiles)")
print("   - row, col are GLOBAL indices (0-1023 for 1024x1024 matrix)")
print("   - Use LOCAL for shared memory, GLOBAL for main memory!")

### Mistake 2: Missing Second syncthreads

In [ ]:
# ❌ WRONG: Missing sync after computation
# for tile_idx in range(num_tiles):
#     sA[ty, tx] = A[row, tile_idx * TILE_SIZE + tx]
#     sB[ty, tx] = B[tile_idx * TILE_SIZE + ty, col]
#     cuda.syncthreads()  # Wait for load
#     for k in range(TILE_SIZE):
#         tmp += sA[ty, k] * sB[k, tx]
#     # Missing syncthreads() here!
#     # Next iteration overwrites shared memory while some threads still reading!

# ✅ CORRECT: Sync after both load AND compute
print("💡 You need TWO syncthreads per tile iteration:")
print("   1. After loading tile (before computation)")
print("   2. After computation (before loading next tile)")
print("   Missing either causes race conditions!")

### Mistake 3: Not Handling Matrix Size Not Divisible by Tile Size

In [ ]:
# Test with non-tile-aligned size
A_odd = np.random.randn(500, 500).astype(np.float32)
B_odd = np.random.randn(500, 500).astype(np.float32)

result_odd = matmul_gpu_tiled(A_odd, B_odd)
expected_odd = A_odd @ B_odd

print(f"Testing 500×500 (not divisible by 16):")
print(f"  Correct: {np.allclose(result_odd, expected_odd, rtol=1e-4)}")
print("\n💡 Our implementation handles this with bounds checking:")
print("   - if row < M and a_col < K: sA[ty, tx] = A[row, a_col]")
print("   - else: sA[ty, tx] = 0.0  # Zero padding")

---

## ✋ Try It Yourself: Implement Larger Tiles

**Challenge:** Modify the tiled kernel to use 32×32 tiles instead of 16×16.

Considerations:
- More data reuse (should be faster)
- Uses more shared memory (check limits!)
- 32×32 = 1024 threads per block (exactly the max on most GPUs)

In [ ]:
# TODO: Implement 32×32 tiled matrix multiplication

TILE_SIZE_32 = 32

@cuda.jit
def matmul_tiled_32_kernel(A, B, C):
    """
    Tiled matmul with 32×32 tiles.
    
    TODO: Implement this!
    
    Hints:
    - Same logic as 16×16 version
    - Change TILE_SIZE constant
    - Make sure shared memory fits (32×32×4 = 4KB per tile, 8KB total)
    """
    # YOUR CODE HERE
    pass


# Test when implemented:
# threads_per_block_32 = (32, 32)
# blocks_32 = (32, 32)  # 1024 / 32 = 32
# matmul_tiled_32_kernel[blocks_32, threads_per_block_32](d_A, d_B, d_C)
# result = d_C.copy_to_host()
# print(f"32×32 tiles correct: {np.allclose(result, C_reference, rtol=1e-4)}")

<details>
<summary>💡 Hint</summary>

The code is nearly identical! Just change:
```python
TILE_SIZE_32 = 32
sA = cuda.shared.array(shape=(32, 32), dtype=float32)
sB = cuda.shared.array(shape=(32, 32), dtype=float32)
```

And launch with:
```python
threads_per_block = (32, 32)
blocks = (1024 // 32, 1024 // 32)
```
</details>

---

## 🎉 Checkpoint

Congratulations! You've learned:

- ✅ **Why matmul is fundamental** - It's 90%+ of neural network compute
- ✅ **Memory access patterns** - Why naive is slow, tiled is fast
- ✅ **Shared memory tiling** - The most important GPU optimization
- ✅ **Bank conflicts** - And how to avoid them
- ✅ **Benchmarking against cuBLAS** - Industry-standard reference

You achieved performance within **2-5x of cuBLAS** with just ~50 lines of CUDA Python!

---

## 🚀 Challenge (Optional)

**Advanced Challenge: Implement Register Blocking**

Instead of each thread computing one output element, have each thread compute a small tile (e.g., 4×4) of the output.

```python
# Each thread has 16 accumulators in registers:
c00, c01, c02, c03 = 0.0, 0.0, 0.0, 0.0
c10, c11, c12, c13 = 0.0, 0.0, 0.0, 0.0
c20, c21, c22, c23 = 0.0, 0.0, 0.0, 0.0
c30, c31, c32, c33 = 0.0, 0.0, 0.0, 0.0

# Load 4 elements of A row, 4 elements of B column into registers
# Compute 16 FMAs per load - better arithmetic intensity!
```

This technique, combined with prefetching, is how cuBLAS achieves near-peak performance.

---

## 📖 Further Reading

- [Volkov's Better Performance at Lower Occupancy](https://www.nvidia.com/content/GTC-2010/pdfs/2238_GTC2010.pdf) - Classic paper on register blocking
- [CUTLASS GEMM Tutorial](https://github.com/NVIDIA/cutlass/blob/master/media/docs/gemm_api.md) - NVIDIA's template library for matmul
- [Why GEMM is at the Heart of Deep Learning](https://petewarden.com/2015/04/20/why-gemm-is-at-the-heart-of-deep-learning/)
- [Tensor Core Programming](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#wmma)

---

## 🧹 Cleanup

In [ ]:
import gc

# Clean up large arrays
if 'A_1024' in dir():
    del A_1024
if 'B_1024' in dir():
    del B_1024
if 'C_reference' in dir():
    del C_reference
if 'A_small_np' in dir():
    del A_small_np
if 'A_small_cp' in dir():
    del A_small_cp
if 'A_odd' in dir():
    del A_odd
if 'B_odd' in dir():
    del B_odd
if 'd_A' in dir():
    del d_A
if 'd_B' in dir():
    del d_B
if 'd_C' in dir():
    del d_C

if HAS_CUPY:
    if 'cp_A' in dir():
        del cp_A
    if 'cp_B' in dir():
        del cp_B
    if 'cp_C' in dir():
        del cp_C
    cp.get_default_memory_pool().free_all_blocks()

gc.collect()

# Clear CUDA context
try:
    cuda.current_context().reset()
except Exception:
    pass  # Context might not exist

print("✅ GPU memory cleared!")
print("\n➡️ Ready for Lab 1.3.3: Custom Embedding Lookup")